# inference komt model with lora

## Install Dependencies 

In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-jrotx6iw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-jrotx6iw
  Resolved https://github.com/huggingface/peft.git to commit 139624750a6a431b9958be5c9485aec5571d64c1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-nq5k1okx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-nq5k1okx
  Resolved https://github.com/huggingface/accelerate.git to commit 629d02c8446354860c9bdf58b6bc006186cbc818
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-h1gtjup7
  Running c

## load model

In [2]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig


In [3]:
model='davidkim205/komt-llama2-7b-v1'
peft_model_name = 'davidkim205/komt-llama2-7b-v1-lora'

In [4]:
config = PeftConfig.from_pretrained(peft_model_name)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
config.base_model_name_or_path =model
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto")
model = PeftModel.from_pretrained(model, peft_model_name)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
streamer = TextStreamer(tokenizer)

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def gen(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"### instruction: {x}\n\n### Response: "
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"\n\n### Response: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


In [6]:
gen('넷플릭스는 어떤 시청 문화를 확산시켰나요?')

</s> ### instruction: 넷플릭스는 어떤 시청 문화를 확산시켰나요?

### Response: 

/home/david/anaconda3/envs/komt/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/david/anaconda3/envs/komt/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


넷플릭스는 매주 1~2편을 공개했던 TV 드라마에 맞서 한번에 전편을 몰아볼 수 있는 '몰아보기' 시청 문화를 확산시켰습니다.</s>


"넷플릭스는 매주 1~2편을 공개했던 TV 드라마에 맞서 한번에 전편을 몰아볼 수 있는 '몰아보기' 시청 문화를 확산시켰습니다.</s>"

In [7]:
gen('잠을 잘자는 방법 알려줘')

</s> ### instruction: 잠을 잘자는 방법 알려줘

### Response: 잠을 잘 자는 방법을 알려드리겠습니다.

1. 매일 일정한 시간에 규칙적으로 일어나세요.
2. 카페인, 니코틴, 알코올과 같이 중추신경계에 작용하는 약물을 섭취하지 않으세요.
3. 매일 같은 시간에 잠자리에 들기
4. 잠자리에 잠든 후 잠을 잘 때는 밝은 환경을 추천합니다.
5. 잠자리는 편안하고 쾌적한 환경이어야 합니다.
6. 잠자리에서 편안하게 느낄 수 있는 방법으로 침대에 누워 자거나 편안한 자세를 취하세요.
7. 잠자리에 들기 1시간 전에 전자기기 사용을 자제하세요.
8. 잠자리에 들기 1시간 전에 밝은 음악, 밝은 빛 등 자극을 제거하세요.
9. 잠자리에 들기 30분 전에 따뜻한 온도의 운동을 하세요.
10. 


'잠을 잘 자는 방법을 알려드리겠습니다.\n\n1. 매일 일정한 시간에 규칙적으로 일어나세요.\n2. 카페인, 니코틴, 알코올과 같이 중추신경계에 작용하는 약물을 섭취하지 않으세요.\n3. 매일 같은 시간에 잠자리에 들기\n4. 잠자리에 잠든 후 잠을 잘 때는 밝은 환경을 추천합니다.\n5. 잠자리는 편안하고 쾌적한 환경이어야 합니다.\n6. 잠자리에서 편안하게 느낄 수 있는 방법으로 침대에 누워 자거나 편안한 자세를 취하세요.\n7. 잠자리에 들기 1시간 전에 전자기기 사용을 자제하세요.\n8. 잠자리에 들기 1시간 전에 밝은 음악, 밝은 빛 등 자극을 제거하세요.\n9. 잠자리에 들기 30분 전에 따뜻한 온도의 운동을 하세요.\n10.'